<a href="https://colab.research.google.com/github/kobosung4756/what-I-learned-today/blob/main/hw5_solarpower_ver1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import random
import tensorflow as tf

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)

if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [ ]:
# device='cpu'

In [ ]:
data=pd.read_csv('train.csv')
data2=pd.read_csv('X_test.csv')
submit=pd.read_csv('sample_submit.csv')

In [ ]:
xy=np.array(data, dtype=np.float32)

train_X = torch.FloatTensor(xy[:,0:-1])
train_Y=torch.FloatTensor(xy[:,[-1]])
# train_Y=torch.squeeze(train_Y, 1)

In [ ]:
print(train_X)
print(train_Y)
print(train_X.shape)
print(train_Y.shape)
print(train_Y.unique())

tensor([[  0.0000,   0.0000,   0.0000,  ...,   1.5000,  69.0800, -12.0000],
        [  0.0000,   0.0000,  30.0000,  ...,   1.5000,  69.0600, -12.0000],
        [  0.0000,   1.0000,   0.0000,  ...,   1.6000,  71.7800, -12.0000],
        ...,
        [208.0000,   6.0000,   0.0000,  ...,   1.1000,  47.4600,  15.0000],
        [208.0000,   6.0000,  30.0000,  ...,   1.4000,  44.5100,  17.0000],
        [208.0000,   7.0000,   0.0000,  ...,   1.7000,  37.8000,  19.0000]])
tensor([[ 0.0000],
        [ 0.0000],
        [ 0.0000],
        ...,
        [13.8872],
        [23.2699],
        [33.0276]])
torch.Size([9999, 8])
torch.Size([9999, 1])
tensor([0.0000e+00, 9.3828e-02, 9.3831e-02,  ..., 9.7288e+01, 9.7667e+01,
        9.7850e+01])


In [ ]:
# 학습 파라미터 설정
learning_rate = 0.001
drop_prob = 0.3

In [ ]:
linear1 = torch.nn.Linear(8,512,bias=True)
linear2 = torch.nn.Linear(512,512,bias=True)
linear3 = torch.nn.Linear(512,512,bias=True)
linear4 = torch.nn.Linear(512,512,bias=True)
linear5 = torch.nn.Linear(512,1,bias=True)
relu = torch.nn.ReLU()
dropout = torch.nn.Dropout(p=drop_prob)

In [ ]:
# Random Init => Xavier Init
torch.nn.init.xavier_normal_(linear1.weight)
torch.nn.init.xavier_normal_(linear2.weight)
torch.nn.init.xavier_normal_(linear3.weight)
torch.nn.init.xavier_normal_(linear4.weight)
torch.nn.init.xavier_normal_(linear5.weight)

Parameter containing:
tensor([[ 1.0177e-01, -3.5125e-02, -2.0119e-02,  2.5088e-02,  5.4480e-02,
         -4.6233e-02, -5.4380e-02, -2.1184e-03, -7.2732e-02,  4.6227e-02,
          2.5965e-02,  5.3083e-02, -3.8221e-03,  2.0609e-02,  7.5596e-03,
          3.9848e-02, -7.2982e-02, -6.5854e-02,  3.4890e-02, -1.8343e-02,
          3.3812e-02, -1.2580e-01,  1.3557e-01, -8.1427e-02, -1.3377e-02,
          3.7150e-02, -4.4006e-02,  9.9063e-03,  5.2414e-02,  1.4235e-02,
          4.1581e-02,  5.3383e-03,  1.4287e-02,  9.7459e-02, -3.8616e-02,
          6.9806e-02,  2.8713e-02,  1.8915e-02, -8.3421e-02,  5.3672e-02,
          3.8063e-02, -3.2953e-02,  1.6238e-02, -1.3073e-02, -2.8187e-03,
         -2.1863e-02,  3.1670e-02, -1.6422e-02,  1.4516e-03,  8.0996e-04,
         -3.0874e-02,  6.1301e-02, -3.1781e-02,  8.3663e-02, -1.2186e-01,
          5.8655e-02, -1.5628e-02, -3.7894e-02, -4.1394e-02,  5.0144e-02,
         -1.4037e-03,  6.4192e-02, -7.0164e-02,  1.7584e-02, -4.7895e-02,
          1.8228

In [ ]:
# ======================================
# relu는 맨 마지막 레이어에서 빼는 것이 좋다.
# ======================================
model = torch.nn.Sequential(linear1,relu,dropout,
                            linear2,relu,dropout,
                            linear3,relu,dropout,
                            linear4,relu,dropout,
                            linear5).to(device)


In [ ]:
# 손실함수와 최적화 함수
loss = torch.nn.MSELoss().to(device) # softmax 내부적으로 계산
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 

In [ ]:
train_X=train_X.to(device)
train_Y=train_Y.to(device)

for stop in range(20000):
    
    # 그래디언트 초기화
    optimizer.zero_grad()
    # Forward 계산
    hypothesis = model(train_X)
    # Error 계산
    cost = loss(hypothesis, train_Y)
    # Backward 계산 
    cost.backward()
    # 가중치 갱신
    optimizer.step()

    if stop % 100 == 0:
        print(stop, cost.item())

0 578.6893920898438
100 49.829471588134766
200 30.076013565063477
300 25.092615127563477
400 23.269973754882812
500 21.561586380004883
600 21.523014068603516
700 20.211715698242188
800 19.287485122680664
900 19.056116104125977
1000 17.732154846191406
1100 17.444103240966797
1200 17.473554611206055
1300 15.96004581451416
1400 14.779253005981445
1500 15.227150917053223
1600 14.3407621383667
1700 14.289031982421875
1800 13.875423431396484
1900 12.83409309387207
2000 11.961563110351562
2100 11.376832962036133
2200 11.257830619812012
2300 12.111675262451172
2400 11.445505142211914
2500 10.903603553771973
2600 9.763093948364258
2700 9.259719848632812
2800 9.470353126525879
2900 9.011978149414062
3000 9.103261947631836
3100 9.478164672851562
3200 8.177597045898438
3300 8.849291801452637
3400 8.75355052947998
3500 7.775436878204346
3600 7.939126014709473
3700 7.931691646575928
3800 7.638484001159668
3900 8.284515380859375
4000 7.368402481079102
4100 7.524023532867432
4200 8.392068862915039
430

In [ ]:
xy2_x=np.array(data2, dtype=np.float32)

test_X = torch.FloatTensor(xy2_x[:,1:])

In [ ]:
print(test_X)

tensor([[1043.0000,    7.0000,   30.0000,  ...,    2.0000,   63.0200,
            3.0000],
        [1043.0000,    8.0000,    0.0000,  ...,    2.3000,   61.5400,
            5.0000],
        [1043.0000,    8.0000,   30.0000,  ...,    2.4000,   57.3400,
            6.0000],
        ...,
        [1094.0000,   22.0000,   30.0000,  ...,    2.2000,   66.7800,
           -4.0000],
        [1094.0000,   23.0000,    0.0000,  ...,    2.1000,   67.7200,
           -4.0000],
        [1094.0000,   23.0000,   30.0000,  ...,    2.1000,   67.7000,
           -4.0000]])


In [ ]:
# Test the model using test sets
test_X=test_X.to(device)
with torch.no_grad():
    model.eval()  # 주의사항 (dropout=False)
    prediction = model(test_X)
    correct_prediction = torch.argmax(prediction, 1)  

In [ ]:
result=correct_prediction.cpu().numpy()
print(result)

[0 0 0 ... 0 0 0]


AttributeError: ignored

In [ ]:
submit['TARGET']=result
submit.to_csv('result_submit.csv',index=False)